In [1]:
# C-2.24

#       Class E-Book
#        _list
#        __getitem__()
#        __len__()
#        __contains__()
#             |
#     ----------------
#    |                \
# Class Purchase     Class Read
#  buy_book()         read_book()

In [2]:
# C-2.25

class Vector:
    def __init__(self, d):
        if isinstance(d, int):
            self._coords = [0] * d
        else:                                  
            try:                                     
                self._coords = [val for val in d]
            except TypeError:
                raise TypeError('invalid parameter type')

    def __len__(self):
        return len(self._coords)

    def __getitem__(self, j):
        return self._coords[j]

    def __setitem__(self, j, val):
        self._coords[j] = val         

    def __str__(self):
        return '<' + str(self._coords)[1:-1] + '>'  
    
    def __mul__(self, other):
        if isinstance(other, (int, float)):
            result = Vector(len(self))
            for j in range(len(self)):
                result[j] = self[j] * other
            return result
        else:
            return sum(self[j]*other[j] for j in range(len(self)))
                
u = Vector([1,2,3])
v = Vector([4,5,6])
k = 3

w = u * v
print(w)
t = u * k
print(str(t))

32
<3, 6, 9>


In [6]:
# C-2.26
class ReversedSequenceIterator:
    '''A reversed iterator for any of Python sequence types'''
    
    def __init__(self, sequence):
        self._seq = sequence
        self._k = 0
        
    def __next__(self):
        self._k -= 1
        if self._k >= -len(self._seq):
            return (self._seq[self._k])
        else:
            raise StopIteration()
            
    def __iter__(self):
        return self
    
x = ReversedSequenceIterator([1,2,3])
print(next(x))
print(next(x))
print(next(x))

3
2
1


In [8]:
# C-2.27
class Range:
    def __init__(self, start, stop=None, step=1):
        if step == 0:
            raise ValueError('step cannot be 0')
        if stop is None:                  
            start, stop = 0, start          
        self._length = max(0, (stop - start + step - 1) // step)
        self._start = start
        self._step = step

    def __len__(self):
        return self._length

    def __getitem__(self, k):
        if k < 0:
            k += len(self)                  
        if not 0 <= k < self._length:
            raise IndexError('index out of range')
        return self._start + k * self._step

    def __contains__(self, val):
        try:
            index = (val - self._start)//self._step
            if self[index] == val:
                return True
        except IndexError:
            return False

print(2 in range(1000000))
print(999999 in range(1000000))
print(10000000 in range(1000000))

True
True
False


In [9]:
# C-2.28
class CreditCard:
    def __init__(self, customer, bank, acnt, limit):
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0

    def get_customer(self):
        return self._customer
    
    def get_bank(self):
        return self._bank

    def get_account(self):
        return self._account

    def get_limit(self):
        return self._limit

    def get_balance(self):
        return self._balance

    def charge(self, price):
        if price + self._balance > self._limit:  
            return False                          
        else:
            self._balance += price
        return True

    def make_payment(self, amount):
        self._balance -= amount

class PredatoryCreditCard(CreditCard):
    MAX_CHARGES = 10
    
    def __init__(self, customer, bank, acnt, limit, apr):
        super().__init__(customer, bank, acnt, limit)  # call super constructor
        self._apr = apr
        self._charges = 0

    def charge(self, price):
        success = super().charge(price)         
        if not success:
            self._balance += 5 
        else:
            self._charges += 1
            if self._charges > self.MAX_CHARGES:
                self._balance += 1     
        return success                           

    def process_month(self):
        if self._balance > 0:
            monthly_factor = pow(1 + self._apr, 1/12)
            self._balance *= monthly_factor
        self._charges = 0 # reset for new month
        
john = PredatoryCreditCard('John Adams', 'USBANK', '3055', 100, 0.02)
for i in range(12):
    john.charge(1)
    if i < 10: 
        print('Monthly charge #'+str(i+1)+'| Balance: $'+str(john.get_balance()))
    else:
        print('Monthly charge #'+str(i+1)+'| Balance: $'+str(john.get_balance())+' ($1 fee added)')

Monthly charge #1| Balance: $1
Monthly charge #2| Balance: $2
Monthly charge #3| Balance: $3
Monthly charge #4| Balance: $4
Monthly charge #5| Balance: $5
Monthly charge #6| Balance: $6
Monthly charge #7| Balance: $7
Monthly charge #8| Balance: $8
Monthly charge #9| Balance: $9
Monthly charge #10| Balance: $10
Monthly charge #11| Balance: $12 ($1 fee added)
Monthly charge #12| Balance: $14 ($1 fee added)


In [10]:
# C-2.29
class CreditCard:
    def __init__(self, customer, bank, acnt, limit):
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0

    def get_customer(self):
        return self._customer
    
    def get_bank(self):
        return self._bank

    def get_account(self):
        return self._account

    def get_limit(self):
        return self._limit

    def get_balance(self):
        return round(self._balance, 2)

    def charge(self, price):
        if price + self._balance > self._limit:  
            return False                          
        else:
            self._balance += price
        return True

    def make_payment(self, amount):
        self._balance -= amount

class PredatoryCreditCard(CreditCard):
    LATE_FEE = 20
    
    def __init__(self, customer, bank, acnt, limit, apr):
        super().__init__(customer, bank, acnt, limit)  # call super constructor
        self._apr = apr
        
    def charge(self, price):
        success = super().charge(price)         
        if not success:
            self._balance += 5  
        self._monthly_payment = self._balance * 0.03
        return success 
    
    def make_monthly_payment(self, amount):
        self._monthly_payment -= amount
    
    def process_month(self):
        if self._monthly_payment > 0:
            self._balance = self._balance + self.LATE_FEE + self._monthly_payment
        if self._balance > 0:
            monthly_factor = pow(1 + self._apr, 1/12)
            self._balance *= monthly_factor
            
x = PredatoryCreditCard('John Adams', 'USBANK', '3509', 1000, 0.03)

x.charge(150) # monthly_payment is $4.5
x.make_monthly_payment(2) # monthly_payment is now $2.5

x.process_month() # Month is processed without completing monthly payment
print(x.get_balance()) # $20 fee added for not completing monthly payment, 
                       # plus, monthly_payment itself and APR interest

172.93


In [11]:
# C-2.30
class CreditCard:
    
    def __init__(self, customer, bank, acnt, limit):
        self._customer = customer
        self._bank = bank
        self._account = acnt
        self._limit = limit
        self._balance = 0

    def get_customer(self):
        return self._customer
    
    def get_bank(self):
        return self._bank

    def get_account(self):
        return self._account

    def get_limit(self):
        return self._limit

    def get_balance(self):
        return self._balance
    
    def _set_balance(self, b): # implement setter method
        self._balance = b

    def charge(self, price):
        if price + self._balance > self._limit:  
            return False                          
        else:
            self._balance += price
        return True

    def make_payment(self, amount):
        self._balance -= amount

class PredatoryCreditCard(CreditCard):
    
    def __init__(self, customer, bank, acnt, limit, apr):
        
        super().__init__(customer, bank, acnt, limit)  
        self._apr = apr
       
    def charge(self, price):
        success = super().charge(price)         
        if not success:
            self._set_balance( self.get_balance()+5 )  # use _set_balance with get_balance to change value
        return success                           

    def process_month(self):
        if self.get_balance() > 0:
            monthly_factor = pow(1 + self._apr, 1/12)
            self._set_balance( round(self.get_balance()*monthly_factor, 2) ) # se _set_balance with get_balance to change value
        
x = PredatoryCreditCard('Tristan Lake', 'USBank', '3509', 1000, 0.04)
x.charge(100)
x.process_month()
x.get_balance()

100.33

In [14]:
# C-2.31
class Progression:
    def __init__(self, start=0):
        self._current = start

    def _advance(self):
        self._current += 1

    def __next__(self):
        if self._current is None:    # our convention to end a progression
            raise StopIteration()
        else:
            answer = self._current     # record current value to return
            self._advance()            # advance to prepare for next time
            return answer              # return the answer

    def __iter__(self):
        return self                  

    def print_progression(self, n):
        print('|'.join(str(next(self)) for j in range(n)))
        
class AbsDiffProgression(Progression):
    
    def __init__(self, val1=2, val2=200):
        super().__init__(val1) # _current is now val1
        self._prev = val2 # _prev is val2

    def _advance(self):
        self._current, self._prev = self._prev, abs(self._current - self._prev)
        

y = AbsDiffProgression()
y.print_progression(10)   

2|200|198|2|196|194|2|192|190|2


In [ ]:
# C-2.32
class Progression:
    def __init__(self, start=0):
        self._current = start

    def _advance(self):
        self._current += 1

    def __next__(self):
        if self._current is None:    # our convention to end a progression
            raise StopIteration()
        else:
            answer = self._current     # record current value to return
            self._advance()            # advance to prepare for next time
            return answer              # return the answer

    def __iter__(self):
        return self                  

    def print_progression(self, n):
        print('|'.join(str(next(self)) for j in range(n)))
        
class SqrtProgression(Progression):
    
    def __init__(self, strt=65536):
        super().__init__(strt)

    def _advance(self):
        self._current = round( (self._current)**(0.5), 3 ) 
        

y = SqrtProgression()
y.print_progression(10)  